In [ ]:
from collections import OrderedDict
from robustness import datasets
from robustness.tools.breeds_helpers import make_entity30
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import sampler
import os
import torchvision
import pandas as pd
# for plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

from trainer_functions.customImageNettrainer import build_resnet50, evaluate_customImageNet, train_customImageNet

In [ ]:
data_dir = '/Users/rada/.mxnet/datasets/imagenet'
info_dir = '/Users/rada/Documents/GitHub/BREEDS-Benchmarks/imagenet_class_hierarchy/modified'

n_subset = 1500
lrs = [0.0005]

ret = make_entity30(info_dir, split="rand")
superclasses, subclass_split, label_map = ret
train_subclasses, test_subclasses = subclass_split

dataset_source = datasets.CustomImageNet(data_dir, train_subclasses)
loaders_source = dataset_source.make_loaders(workers=10, batch_size=64, shuffle_train=64, shuffle_val=64, subset=n_subset)
train_loader_source, val_loader_source = loaders_source

dataset_target = datasets.CustomImageNet(data_dir, test_subclasses)
loaders_target = dataset_target.make_loaders(workers=10, batch_size=64, shuffle_train=True, shuffle_val=True, subset=n_subset)
train_loader_target, val_loader_target = loaders_target


In [ ]:
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

for lr in lrs:
    device = torch.device("mps")
    tune_net = build_resnet50(device)

    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=3)
    
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=2)

    # the first layer is fine-tuned
    optimizer = optim.Adam(tune_net.layer1.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_target, val_loader_target, optimizer, scheduler, device=device, epochs=15)
    
    # adding results to dataframe
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "first layer"  # change when run
    dict['n_subset'] = n_subset
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp])

# change when run
df.to_csv("Entity30_layer1.csv")

Building model...
Epoch 0, loss = 3.2286, lr 0.00100000
Got 919 / 2000 correct (45.95)

Epoch 1, loss = 2.0393, lr 0.00100000
Got 973 / 2000 correct (48.65)

Epoch 2, loss = 2.8932, lr 0.00100000
Got 1017 / 2000 correct (50.85)

Epoch 3, loss = 3.0702, lr 0.00100000
Got 1005 / 2000 correct (50.25)

Epoch 4, loss = 1.9675, lr 0.00100000
Got 1008 / 2000 correct (50.40)

Epoch 5, loss = 1.7241, lr 0.00100000
Got 994 / 2000 correct (49.70)

Epoch 6, loss = 2.6093, lr 0.00100000
Got 1027 / 2000 correct (51.35)

Epoch 7, loss = 2.2146, lr 0.00100000
Got 1011 / 2000 correct (50.55)

Epoch 8, loss = 2.1076, lr 0.00100000
Got 1000 / 2000 correct (50.00)

Epoch 9, loss = 1.3700, lr 0.00100000
Got 1006 / 2000 correct (50.30)

Building model...
Epoch 0, loss = 3.1209, lr 0.00010000
Got 911 / 2000 correct (45.55)

Epoch 1, loss = 3.2269, lr 0.00010000
Got 974 / 2000 correct (48.70)

Epoch 2, loss = 3.4049, lr 0.00010000
Got 978 / 2000 correct (48.90)

Epoch 3, loss = 3.6676, lr 0.00010000
Got 993 /

In [ ]:
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

for lr in lrs:
    device = torch.device("mps")
    tune_net = build_resnet50(device)

    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=3)
    
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=2)

    # the first layer is fine-tuned
    optimizer = optim.Adam(tune_net.layer2.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_target, val_loader_target, optimizer, scheduler, device=device, epochs=15)
    
    # adding results to dataframe
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "second layer"  # change when run
    dict['n_subset'] = n_subset
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp])

# change when run
df.to_csv("Entity30_layer2.csv")

In [ ]:
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

for lr in lrs:
    device = torch.device("mps")
    tune_net = build_resnet50(device)

    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=3)
    
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=2)

    # the first layer is fine-tuned
    optimizer = optim.Adam(tune_net.layer3.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_target, val_loader_target, optimizer, scheduler, device=device, epochs=15)
    
    # adding results to dataframe
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "third layer"  # change when run
    dict['n_subset'] = n_subset
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp])

# change when run
df.to_csv("Entity30_layer3.csv")

In [ ]:
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

for lr in lrs:
    device = torch.device("mps")
    tune_net = build_resnet50(device)

    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=3)
    
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=2)

    # the first layer is fine-tuned
    optimizer = optim.Adam(tune_net.layer4.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_target, val_loader_target, optimizer, scheduler, device=device, epochs=15)
    
    # adding results to dataframe
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "fourth layer"  # change when run
    dict['n_subset'] = n_subset
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp])

# change when run
df.to_csv("Entity30_layer4.csv")

In [ ]:
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

for lr in lrs:
    device = torch.device("mps")
    tune_net = build_resnet50(device)

    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=3)
    
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=2)

    # the first layer is fine-tuned
    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_target, val_loader_target, optimizer, scheduler, device=device, epochs=15)
    
    # adding results to dataframe
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "last layer"  # change when run
    dict['n_subset'] = n_subset
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp])

# change when run
df.to_csv("Entity30_last_layer.csv")

In [ ]:
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

for lr in lrs:
    device = torch.device("mps")
    tune_net = build_resnet50(device)

    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=3)
    
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=2)

    # the first layer is fine-tuned
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_target, val_loader_target, optimizer, scheduler, device=device, epochs=15)
    
    # adding results to dataframe
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "all layers"  # change when run
    dict['n_subset'] = n_subset
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp])

# change when run
df.to_csv("Entity30_all_layers.csv")